In [2]:
import pandas as pd
import numpy as np

In [3]:
import os
print(os.getcwd())

e:\Lab\code\Combine_Result_DAST_Tools\Report


Load EXPECTED RESULT

In [4]:
expected_result = pd.read_csv('expectedresults-1.2.csv')

In [5]:
expected_result = expected_result[['# test name', ' category', ' real vulnerability']]

In [6]:
expected_result[' category'].unique()

array(['pathtraver', 'hash', 'trustbound', 'crypto', 'cmdi', 'sqli',
       'weakrand', 'ldapi', 'xss', 'securecookie', 'xpathi'], dtype=object)

In [7]:
pathtraversal = expected_result[expected_result[' category'] == 'pathtraver']
cmdi = expected_result[expected_result[' category'] == 'cmdi']
sqli = expected_result[expected_result[' category'] == 'sqli']
ldapi = expected_result[expected_result[' category'] == 'ldapi']
xss = expected_result[expected_result[' category'] == 'xss']
securecookie = expected_result[expected_result[' category'] == 'securecookie']

LOAD REPORT

In [8]:
wapiti_rp = pd.read_json('./Raw/Wapiti_report/path.json')
zap_rp = pd.read_json('./Raw/ZAP_report/path.json')
merge_rp = pd.read_json('./Combine_report/merged_sarif.json')

In [9]:
vul_merge = merge_rp['runs']
vul_zap = zap_rp['runs']
vul_wapiti = wapiti_rp['vulnerabilities']['Reflected Cross Site Scripting']

In [10]:
merge_errors = []
sever_errors = []
for run in vul_merge:
    for result in run.get('results', []):
        if result.get('ruleId') != 'wapiti-internal-server-error':
            if len(result.get('locations', [{}])[0].get('physicalLocation', {}).get('artifactLocation', {}).get('uri')) != 2:
                url = result.get('locations', [{}])[0].get('physicalLocation', {}).get('artifactLocation', {}).get('uri')
                cond = url.split("/")[-1].split("?")[0]
                merge_errors.append(cond)
            else:
                url = result.get('locations', [{}])[0].get('physicalLocation', {}).get('artifactLocation', {}).get('uri')[1]
                cond = url.split(" ")[2].split("/")[-1].split("?")[0]
                merge_errors.append(cond)
        else:
            url = result.get('locations', [{}])[0].get('physicalLocation', {}).get('artifactLocation', {}).get('uri')
            cond = url.split("/")[-1].split("?")[0]
            sever_errors.append(cond)

print("Số lượng không phải 'wapiti-internal-server-error':", len(merge_errors))
print("Số lượng sever_errors:", len(sever_errors))
print("Số lượng sever_errors unique:", len(set(sever_errors)))
print("Số lượng merge_errors unique:", len(set(merge_errors + sever_errors)))

Số lượng không phải 'wapiti-internal-server-error': 72
Số lượng sever_errors: 5
Số lượng sever_errors unique: 5
Số lượng merge_errors unique: 72


In [28]:
zap_errors = []
for run in vul_zap:
    for result in run.get('results', []):
        url = result.get('locations', [{}])[0].get('physicalLocation', {}).get('artifactLocation', {}).get('uri')
        cond = url.split("/")[-1].split("?")[0]
        zap_errors.append(cond)

print("Số lượng không phải 'wapiti-internal-server-error':", len(zap_errors))
print("Số lượng zap_errors unique:", len(set(zap_errors)))

Số lượng không phải 'wapiti-internal-server-error': 71
Số lượng zap_errors unique: 68


In [12]:
wapiti_errors = []
for result in vul_wapiti:
    url = result.get('path')
    cond = url.split("/")[-1]
    wapiti_errors.append(cond)
print("Số lượng Command execution của Wapiti:", len(wapiti_errors))
print("Số lượng wapiti_errors unique:", len(set(wapiti_errors)))

Số lượng Command execution của Wapiti: 138
Số lượng wapiti_errors unique: 138


In [12]:
wapitit = list(set(wapiti_errors)) + list(set(sever_errors))

COMPARE

In [13]:
tmp = xss.copy()

In [14]:
print(xss[' real vulnerability'].value_counts())

 real vulnerability
True     246
False    209
Name: count, dtype: int64


In [15]:
tmp["predicted"] = 0

In [16]:
tmp

,# test name,category,real vulnerability,predicted
12,BenchmarkTest00013,xss,True,0
13,BenchmarkTest00014,xss,True,0
29,BenchmarkTest00030,xss,True,0
35,BenchmarkTest00036,xss,True,0
40,BenchmarkTest00041,xss,True,0
...,...,...,...,...
2692,BenchmarkTest02693,xss,False,0
2693,BenchmarkTest02694,xss,False,0
2694,BenchmarkTest02695,xss,False,0
2695,BenchmarkTest02696,xss,False,0


In [21]:
zap_unique = list(set(zap_errors))
len(zap_unique)

161

In [17]:
tmp['predicted'] = tmp['# test name'].apply(lambda x: 1 if x in merge_errors else 0)

In [18]:
filetam = tmp[tmp['predicted'] == 1]['# test name'].tolist()

In [27]:
ans = []
for i in zap_unique:
    if i in filetam:
        # Do something
        continue
    else:
        ans.append(i)
print(len(ans))

48


In [28]:
ans

['BenchmarkTest01011;',
 'BenchmarkTest01882;',
 'BenchmarkTest01883;',
 'BenchmarkTest01885;',
 'BenchmarkTest00109;',
 'BenchmarkTest00996;',
 'BenchmarkTest01890;',
 'BenchmarkTest01000;',
 'BenchmarkTest00107;',
 'BenchmarkTest01012;',
 'BenchmarkTest01880;',
 'BenchmarkTest00998;',
 'BenchmarkTest01887;',
 'BenchmarkTest00104;',
 'BenchmarkTest01001;',
 'BenchmarkTest00100;',
 'BenchmarkTest00113;',
 'BenchmarkTest01888;',
 'BenchmarkTest00997;',
 'BenchmarkTest01884;',
 'BenchmarkTest00112;',
 'BenchmarkTest01886;',
 'BenchmarkTest01879;',
 'BenchmarkTest01889;',
 'BenchmarkTest00105;',
 'BenchmarkTest00101;',
 'BenchmarkTest01007;',
 'BenchmarkTest01878;',
 'BenchmarkTest01009;',
 'BenchmarkTest01002;',
 'BenchmarkTest01891;',
 'BenchmarkTest00108;',
 'BenchmarkTest00114;',
 'BenchmarkTest01006;',
 'BenchmarkTest00110;',
 'BenchmarkTest01010;',
 'BenchmarkTest01008;',
 'BenchmarkTest00102;',
 'BenchmarkTest00106;',
 'BenchmarkTest00103;',
 'BenchmarkTest01881;',
 'BenchmarkTest0

In [19]:
tmp[tmp['predicted']==1]

,# test name,category,real vulnerability,predicted
29,BenchmarkTest00030,xss,True,1
40,BenchmarkTest00041,xss,True,1
46,BenchmarkTest00047,xss,True,1
47,BenchmarkTest00048,xss,True,1
48,BenchmarkTest00049,xss,True,1
...,...,...,...,...
2590,BenchmarkTest02591,xss,True,1
2594,BenchmarkTest02595,xss,True,1
2597,BenchmarkTest02598,xss,True,1
2599,BenchmarkTest02600,xss,True,1


In [20]:
TP = ((tmp[' real vulnerability'] == 1) & (tmp['predicted'] == 1)).sum()
FP = ((tmp[' real vulnerability'] == 0) & (tmp['predicted'] == 1)).sum()
TN = ((tmp[' real vulnerability'] == 0) & (tmp['predicted'] == 0)).sum()
FN = ((tmp[' real vulnerability'] == 1) & (tmp['predicted'] == 0)).sum()

print(f"TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}")

TP: 138, FP: 0, TN: 209, FN: 108


In [218]:
params = [ "BenchmarkTest00041", "BenchmarkTest00047", "BenchmarkTest00048", "BenchmarkTest00049", "BenchmarkTest00382", "BenchmarkTest00387", "BenchmarkTest00467", "BenchmarkTest00472", "BenchmarkTest00477", "BenchmarkTest00492", "BenchmarkTest00644", "BenchmarkTest00719", "BenchmarkTest00724", "BenchmarkTest00729", "BenchmarkTest00800", "BenchmarkTest00801", "BenchmarkTest00802", "BenchmarkTest00803", "BenchmarkTest00804", "BenchmarkTest00805", "BenchmarkTest00806", "BenchmarkTest00807", "BenchmarkTest00809", "BenchmarkTest00810", "BenchmarkTest00811", "BenchmarkTest00822", "BenchmarkTest01253", "BenchmarkTest01258", "BenchmarkTest01263", "BenchmarkTest01268", "BenchmarkTest01506", "BenchmarkTest01511", "BenchmarkTest01596", "BenchmarkTest01657", "BenchmarkTest01658", "BenchmarkTest01660", "BenchmarkTest01662", "BenchmarkTest01665", "BenchmarkTest01667", "BenchmarkTest01670", "BenchmarkTest02123", "BenchmarkTest02133", "BenchmarkTest02223", "BenchmarkTest02228", "BenchmarkTest02402", "BenchmarkTest02487", "BenchmarkTest02578", "BenchmarkTest02579", "BenchmarkTest02580", "BenchmarkTest02582", "BenchmarkTest02583", "BenchmarkTest02584", "BenchmarkTest02586", "BenchmarkTest02587", "BenchmarkTest02588", "BenchmarkTest02591", "BenchmarkTest02595", "BenchmarkTest02598", "BenchmarkTest02600", "BenchmarkTest02608", "BenchmarkTest00030", "BenchmarkTest00375", "BenchmarkTest00376", "BenchmarkTest00378", "BenchmarkTest00380", "BenchmarkTest00383", "BenchmarkTest00384", "BenchmarkTest00385", "BenchmarkTest00388", "BenchmarkTest00390", "BenchmarkTest00395", "BenchmarkTest00473", "BenchmarkTest00475", "BenchmarkTest00478", "BenchmarkTest00642", "BenchmarkTest00643", "BenchmarkTest00645", "BenchmarkTest00651", "BenchmarkTest00656", "BenchmarkTest00711", "BenchmarkTest00715", "BenchmarkTest00720", "BenchmarkTest00721", "BenchmarkTest00725", "BenchmarkTest00728", "BenchmarkTest00737", "BenchmarkTest01254", "BenchmarkTest01257", "BenchmarkTest01259", "BenchmarkTest01260", "BenchmarkTest01261", "BenchmarkTest01266", "BenchmarkTest01267", "BenchmarkTest01284", "BenchmarkTest01335", "BenchmarkTest01337", "BenchmarkTest01346", "BenchmarkTest01347", "BenchmarkTest01349", "BenchmarkTest01350", "BenchmarkTest01505", "BenchmarkTest01507", "BenchmarkTest01509", "BenchmarkTest01510", "BenchmarkTest01583", "BenchmarkTest01584", "BenchmarkTest01587", "BenchmarkTest01589", "BenchmarkTest01590", "BenchmarkTest01592", "BenchmarkTest01594", "BenchmarkTest01597", "BenchmarkTest01598", "BenchmarkTest02126", "BenchmarkTest02127", "BenchmarkTest02129", "BenchmarkTest02130", "BenchmarkTest02132", "BenchmarkTest02136", "BenchmarkTest02145", "BenchmarkTest02224", "BenchmarkTest02225", "BenchmarkTest02227", "BenchmarkTest02232", "BenchmarkTest02234", "BenchmarkTest02241", "BenchmarkTest02396", "BenchmarkTest02399", "BenchmarkTest02400", "BenchmarkTest02403", "BenchmarkTest02405", "BenchmarkTest02409", "BenchmarkTest02410", "BenchmarkTest02480", "BenchmarkTest02483", "BenchmarkTest02489", "BenchmarkTest02493"]

In [219]:
len(params)

137

In [221]:
ans = 0
for i in wapiti_errors:
    if i in params:
        ans += 1
    else:
        print(i)
print(ans)

BenchmarkTest00392
137


In [166]:
tahphop = ["10010","10011","10054"]
merge_errors = []
for run in vul_merge:
    for result in run.get('results', []):
        if result.get('ruleId') in tahphop:
            if len(result.get('locations', [{}])[0].get('physicalLocation', {}).get('artifactLocation', {}).get('uri')) != 2:
                url = result.get('locations', [{}])[0].get('physicalLocation', {}).get('artifactLocation', {}).get('uri')
                cond = url.split("/")[-1].split("?")[0]
                merge_errors.append(cond)
            else:
                url = result.get('locations', [{}])[0].get('physicalLocation', {}).get('artifactLocation', {}).get('uri')[1]
                cond = url.split(" ")[2].split("/")[-1].split("?")[0]
                merge_errors.append(cond)

print("Số lượng không phải 'wapiti-internal-server-error':", len(merge_errors))
print("Số lượng merge_errors unique:", len(set(merge_errors)))
merge_errors = list(set(merge_errors))

Số lượng không phải 'wapiti-internal-server-error': 46
Số lượng merge_errors unique: 40


In [167]:
merge_errors

['BenchmarkTest01280',
 'BenchmarkTest02710',
 'BenchmarkTest01682',
 'BenchmarkTest00977',
 'BenchmarkTest00736',
 'BenchmarkTest00087',
 'BenchmarkTest01861',
 'BenchmarkTest01134',
 'BenchmarkTest01281',
 'BenchmarkTest02507',
 'BenchmarkTest00820',
 'BenchmarkTest00565',
 'BenchmarkTest01863',
 'BenchmarkTest00300',
 'BenchmarkTest02339',
 'BenchmarkTest01862',
 'BenchmarkTest00821',
 'BenchmarkTest00903',
 'BenchmarkTest00241',
 'BenchmarkTest02709',
 'BenchmarkTest00089',
 'BenchmarkTest00169',
 'BenchmarkTest01683',
 'BenchmarkTest01283',
 'BenchmarkTest01789',
 'BenchmarkTest00348',
 'BenchmarkTest02005',
 'BenchmarkTest01282',
 'BenchmarkTest01185',
 'BenchmarkTest01521',
 'BenchmarkTest00491',
 'BenchmarkTest02142',
 'BenchmarkTest00403',
 'BenchmarkTest01061',
 'BenchmarkTest01186',
 'BenchmarkTest02427',
 'BenchmarkTest01187',
 'BenchmarkTest00088',
 'BenchmarkTest00170',
 'BenchmarkTest00566']

In [183]:
vul_wapiti

[]

In [31]:
ans = 0
for i in range(len(merge_errors) - 1):
    for j in range(i+1, len(merge_errors)):
        if merge_errors[i] == merge_errors[j]:
            print(f"Duplicate error found: {merge_errors[i]}")
            ans += 1
print(ans)

Duplicate error found: BenchmarkTest01689
Duplicate error found: BenchmarkTest00409
Duplicate error found: BenchmarkTest00480
Duplicate error found: BenchmarkTest00496
Duplicate error found: BenchmarkTest00498
Duplicate error found: BenchmarkTest00731
Duplicate error found: BenchmarkTest00740
Duplicate error found: BenchmarkTest01270
Duplicate error found: BenchmarkTest01286
Duplicate error found: BenchmarkTest01287
Duplicate error found: BenchmarkTest01362
Duplicate error found: BenchmarkTest01517
Duplicate error found: BenchmarkTest01533
Duplicate error found: BenchmarkTest02137
Duplicate error found: BenchmarkTest02150
Duplicate error found: BenchmarkTest02151
Duplicate error found: BenchmarkTest02152
Duplicate error found: BenchmarkTest02155
Duplicate error found: BenchmarkTest02244
Duplicate error found: BenchmarkTest02251
Duplicate error found: BenchmarkTest02411
Duplicate error found: BenchmarkTest02414
Duplicate error found: BenchmarkTest02429
Duplicate error found: BenchmarkTe